# Análisis 

El punto central de nuestro análisis se centra en ver si las marcas ofrecen variedad de tallas, en especial las tallas 42 y 44 las cuales son las más utilizadas en españa y se corresponden a una talla L/XL. Observemos la variedad y cantidad de tallas que ofrece cada tienda.

In [45]:
import psycopg2
import pandas as pd
import sys
import os
import plotly.express as px  # type: ignore
import plotly.graph_objects as go # type: ignore
from plotly.subplots import make_subplots # type: ignore
import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.path.abspath('../'))
from src import funciones_bbdd as fbd


In [3]:
# Establecemos la conexión
nombre_bbdd = "cuerpos_reales"
conexion = fbd.establecer_conn(nombre_bbdd, "admin", "postgres")

Lo primero es ver cuantos vestidos tenemos de cada tienda para hacernos una idea de los datos con los que estamos trabajando.

In [6]:
cursor = conexion.cursor()
cursor.execute(""" SELECT nombre, talla, num_tallas
FROM (SELECT id_marca, talla, count(talla) AS num_tallas
		FROM vestidos v
		GROUP BY id_marca, talla
		ORDER BY id_marca) AS taux
INNER JOIN marcas AS m ON m.id_marca = taux.id_marca; """)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]

# Crea el DataFrame con los nombres de las columnas
df = pd.DataFrame(data, columns=column_names)

In [11]:
df_mango = df[df["nombre"]=="Mango"]
df_f21 = df[df["nombre"]=="Forever21"]
df_bylila = df[df["nombre"]=="Natural by Lila"]
df_ladypipa = df[df["nombre"]=="Ladypipa"]

In [24]:
talla_orden = ['XXS', 'XS', 'S', 'M', 'L', 'XL', 'XXL', '3XL', '4XL', '5XL']

df_mango['talla'] = pd.Categorical(df_mango['talla'], categories=talla_orden, ordered=True)
df_mango = df_mango.sort_values('talla')

df_f21['talla'] = pd.Categorical(df_f21['talla'], categories=talla_orden, ordered=True)
df_f21 = df_f21.sort_values('talla')

df_bylila['talla'] = pd.Categorical(df_bylila['talla'], categories=talla_orden, ordered=True)
df_bylila = df_bylila.sort_values('talla')

df_ladypipa['talla'] = pd.Categorical(df_ladypipa['talla'], categories=talla_orden, ordered=True)
df_ladypipa = df_ladypipa.sort_values('talla')

In [60]:
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=("Mango", "Forever21", "Natural by Lila", "Ladypipa"))
fig.add_trace(go.Bar(x=df_mango["talla"], y=df_mango["num_tallas"], text=df_mango["num_tallas"],textposition='outside'),row=1, col=1)
fig.add_trace(go.Bar(x=df_f21["talla"], y=df_f21["num_tallas"], text=df_f21["num_tallas"],textposition='outside'),row=1, col=2)
fig.add_trace(go.Bar(x=df_bylila["talla"], y=df_bylila["num_tallas"], text=df_bylila["num_tallas"],textposition='outside'),row=2, col=1)
fig.add_trace(go.Bar(x=df_ladypipa["talla"], y=df_ladypipa["num_tallas"], text=df_ladypipa["num_tallas"],textposition='outside'),row=2, col=2)


# Ajustar el layout de la figura
fig.update_layout(
    title="Distribución de Prendas por Talla",
    margin=dict(t=60)
)

# Establecer el rango del eje y para todas las subgráficas
fig.update_yaxes(range=[0, 200])

# Mostrar la figura
fig.show()